# Assignment 7: Automated Machine Learning (Part 2)
## Objective:

As we learned from the class, the high demand for machine learning has produced a large amount of data scientists who have developed expertise in tools and algorithms. The features in the data will directly influence the results. However, it is tedious and unscalable to manually design and select features without domain knowledge. Thus, using some AutoML techniques will significantly help data scientists save labour and time. 
After completing this assignment, you should be able to answer the following questions:

1. Why do we need AutoML?
2. How does auto feature generation work?
3. How to use featuretools library to automatically generate features?
4. How to get useful features in a large feature space?

Imagine you are a data scientist in an online retailer company, for example, Amazon. Your task is to provide some recommendations to customers based on their historical purchase records.

In this assignment, we predict whether the customer will buy **Banana** in the next 4 weeks. It is a classification problem. To simplify the problem, we have already generated some features and provide the accuracy of the model (Random Forest model). The task for you is to generate **10** useful features and beat our model performance (AUC = 0.61, see below). 

For example, <br>
`MODE(orders.MODE(order_products.product_name)) = Bag of Organic Bananas` means whether the most frequent purchase of the customer is Bag of Organic Bananas. 

```
1: Feature: MODE(orders.MODE(order_products.product_name)) = Bag of Organic Bananas
2: Feature: MODE(order_products.aisle_id) is unknown
3: Feature: SUM(orders.NUM_UNIQUE(order_products.product_name))
4: Feature: MODE(orders.MODE(order_products.product_name)) = Boneless Skinless Chicken Breasts
5: Feature: MODE(order_products.product_name) = Boneless Skinless Chicken Breasts
6: Feature: STD(orders.NUM_UNIQUE(order_products.aisle_id))
7: Feature: MODE(order_products.aisle_id) = 83
8: Feature: MEDIAN(orders.MINUTE(order_time))
9: Feature: MODE(orders.DAY(order_time)) = 23
10: Feature: MODE(orders.MODE(order_products.department)) = produce

AUC 0.61
```


## Preliminary
If you never use featuretools before, you need to learn some basic knowledge of this topic. 
I found that these are some good resources: 
* [featuretools documentation](https://docs.featuretools.com/en/stable/)
* [Tutorial: Automated Feature Engineering in Python](https://towardsdatascience.com/automated-feature-engineering-in-python-99baf11cc219)

The data can be downloaded from [A7-2-data.zip](A7-2-data.zip). 

## 0. Preparation
Import relevant libraries and load the dataset: <br>
users: <br>
* user_id: customer identifier
* label:  1 if the customer will buy banana in next 4 weeks, 0 otherwise

orders: <br>
* order_id: order identifier
* user_id: customer identifier
* order_time: date of the order was placed on 

order_products: <br>
* order_id: order identifier
* order_product_id: foreign key
* reordered:  1 if this product has been ordered by this user in the past, 0 otherwise
* product_name: name of the product
* aisle_id: aisle identifier
* department: the name of the department
* order_time: date of the order was placed on

In [2]:
import pandas as pd

import featuretools as ft
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import os
ft.__version__

'0.13.3'

In [4]:
orders = pd.read_csv("orders.csv")
order_products = pd.read_csv("order_products.csv")
users = pd.read_csv("users.csv")

print(users["label"].value_counts())
orders.shape

False    628
True     139
Name: label, dtype: int64


(5997, 4)

## Task 1. Feature Generation
In this task, you need to use featuretools to generate candidate features by using the above three tables.

### 1.1 Representing Data with EntitySet

Define entities and their relationships (see [https://docs.featuretools.com/en/stable/generated/featuretools.EntitySet.html](https://docs.featuretools.com/en/stable/generated/featuretools.EntitySet.html))

In [5]:
# Get the relationship between entities
def load_entityset(orders, order_products, users):
    entities = {
        "users" : (users.iloc[:,1:], "user_id"),
        "orders" : (orders.iloc[:,1:], "order_id"),
        "order_products" : (order_products.iloc[:,1:], "order_product_id")
    }

    relationships = [("users", "user_id", "orders", "user_id"), 
                     ("orders", "order_id", "order_products", "order_id")]

    return ft.EntitySet("my-entity-set", entities, relationships)

### 1.2 Deep Feature Synthesis

In [6]:
# Automatically generate features
es = load_entityset(orders, order_products, users)

# use ft.dfs to perform feature engineering
feature_matrix, feature_defs = ft.dfs(entityset=es, target_entity='users')

In [14]:
# output what features you generate
feature_matrix

,label,COUNT(orders),SUM(order_products.aisle_id),SUM(order_products.reordered),STD(order_products.aisle_id),STD(order_products.reordered),MAX(order_products.aisle_id),MAX(order_products.reordered),SKEW(order_products.aisle_id),SKEW(order_products.reordered),...,NUM_UNIQUE(orders.MONTH(order_time)),NUM_UNIQUE(orders.MODE(order_products.department)),MODE(orders.DAY(order_time)),MODE(orders.WEEKDAY(order_time)),MODE(orders.YEAR(order_time)),MODE(orders.MODE(order_products.product_name)),MODE(orders.MONTH(order_time)),MODE(orders.MODE(order_products.department)),NUM_UNIQUE(order_products.orders.user_id),MODE(order_products.orders.user_id)
user_id,,,,,,,,,,,,,,,,,,,,,
1,False,4,1271,11,38.070486,0.511766,121,1,0.339280,-0.102843,...,3,1,1,3,2015,Aged White Cheddar Popcorn,1,snacks,1,1
2,True,7,5419,29,39.596448,0.476918,123,1,0.141339,0.682090,...,3,3,1,6,2015,Baked Organic Sea Salt Crunchy Pea Snack,1,produce,1,2
3,False,5,2831,14,40.594305,0.480091,123,1,-0.029453,0.694312,...,3,3,1,6,2015,100% Recycled Paper Towels,1,produce,1,3
7,False,4,4781,36,35.379342,0.503413,123,1,0.172950,0.027978,...,3,1,1,1,2015,85% Lean Ground Beef,3,beverages,1,7
10,False,4,7677,20,37.683787,0.382021,123,1,-0.142492,1.729523,...,2,1,1,5,2015,85% Lean Ground Beef,2,produce,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,False,4,2372,18,44.964947,0.495434,128,1,0.220281,0.422463,...,2,2,1,1,2015,All Natural Maple Almond Butter,1,snacks,1,996
997,False,4,2121,7,40.314601,0.415149,129,1,0.290248,1.476346,...,3,3,1,3,2015,85% Lean Ground Beef,1,produce,1,997
998,False,7,3898,41,37.087147,0.469102,116,1,-0.068441,-0.808589,...,3,2,6,1,2015,Bag of Organic Bananas,1,dairy eggs,1,998


## Task 2. Feature Selection
In this task, you are going to select 10 features that are useful and train the *Random Forest* model. The goal is to beat the accuracy performance as we have shown before. Note that you have to use the Random Forest and the hyperparameters we provide in Section 2.2. In other words, your job is to achieve a higher AUC than 0.61 through feature generation/selection rather than through hyperparameter tuning or model selectoin. 

### 2.1 Select top features

In [8]:
# Select top-10 features and return X, y (X.shape = (767, 10)
df_base = feature_matrix.reset_index()

# get y [will by banana or not]
y = df_base.iloc[:,1]

# compute correlation matrix 
df_corr = df_base.corr(method='spearman')

# get correlation scores with label column
df_corr_label = pd.DataFrame(df_corr["label"].dropna()).reset_index()

# rename column
df_corr_label.rename(columns={'index': 'feature', 'label': 'score'}, inplace=True)

# get top 10 most correlated columns with column label
feature_names = list(df_corr_label.sort_values(by='score', ascending=False).iloc[1:11,0])

# put data of selected features in X dataframe
X = df_base[feature_names]

# print shape of X
print(X.shape)

(767, 10)


### 2.2 Get accuracy and list features

In [10]:
clf = RandomForestClassifier(n_estimators=400, n_jobs=-1)
scores = cross_val_score(estimator=clf,X=X, y=y, cv=3,
                             scoring="roc_auc", verbose=True)

print("AUC %.2f" % (scores.mean()))

print('\n')

# Print top-10 features
for feature in feature_names:
    print(feature)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


AUC 0.66


MEAN(orders.COUNT(order_products))
MEAN(orders.NUM_UNIQUE(order_products.product_name))
MEAN(orders.SUM(order_products.aisle_id))
COUNT(order_products)
SUM(orders.NUM_UNIQUE(order_products.product_name))
SUM(order_products.aisle_id)
NUM_UNIQUE(order_products.product_name)
MIN(orders.NUM_UNIQUE(order_products.product_name))
MIN(orders.COUNT(order_products))
MEAN(orders.NUM_UNIQUE(order_products.department))


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.3s finished


## Task3. Writing Questions

1. Please list three advantages and disadvantages of featuretools. 
2. For those disadvantages you listed above, do you have any idea to improve it? 

<b>1. Advantages & Disadvantages:</b> 

<i>Advantages:</i> 
- Time required to generate features is very low in comparison with manual feature generation. 
- Numerous feature generation (more than 100) for only 3 entities using transformation and aggregation functions. 
- Generation of interpretable features [up to any depth of entities] that could be very hard to do manually. 

<i>Disadvantages:</i>
- DFS does not work for one entity alone and there should be at least two entities to generate features. 
- Some generated features may be useless and does not make any sense. 
- Big number of features generated may lead to poor training of the model, unless we have lots of data. 

<b>2. Solution:</b> 
- We can add one entity to the entityset twice under different names in order to generate features for that unique entity. 
- We need to follow some feature selection methods (filter methods, wrapper methods, embedded methods) to find and choose best features. 
- Amount of data required to train the model is exponential to the number of features. 
so, we need to select only a portion of useful features required to train the model and avoid using all of the features. 
or, we have access to a large amount data based on the number of features. 


## Submission
Complete the code in this notebook, and submit it to the CourSys activity Assignment 7.